In [1]:
library(caret, quiet=TRUE);
library(base64enc)
library(httr, quiet=TRUE)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:
set.seed(1960)

create_model  =  function() {

    model  <- preProcess(iris[, -5], method = c("ica"), n.comp=3)
    
    return(model)
}

In [3]:
# dataset
model = create_model()

In [4]:
pred <- predict(model, as.matrix(iris[, -5]))
head(pred)

ICA1,ICA2,ICA3
-0.1747727812,0.4577064,-1.371333
-0.2411582808,-0.7820956,-1.298537
0.4278364609,-0.3034750,-1.334651
0.5890002061,-0.5442090,-1.271123
0.1680286252,0.6994855,-1.377995
0.0005664192,1.5671793,-1.204708


In [5]:
print(model)

Created from 150 samples and 4 variables

Pre-processing:
  - centered (4)
  - independent component signal extraction (4)
  - ignored (0)
  - scaled (4)

ICA used 3 components


# SQL Code Generation

In [6]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "caret_rpart_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [7]:
lModelSQL = test_ws_sql_gen(model)

In [8]:
cat(lModelSQL)


WITH "CenteredDataForFastICA" AS 
(SELECT "ADS"."KEY" AS "KEY", (CAST("ADS"."Feature_0" AS FLOAT) - 5.843333333333334) / 0.8280661279778629 AS "Feature_0", (CAST("ADS"."Feature_1" AS FLOAT) - 3.0573333333333332) / 0.4358662849366982 AS "Feature_1", (CAST("ADS"."Feature_2" AS FLOAT) - 3.758) / 1.7652982332594664 AS "Feature_2", (CAST("ADS"."Feature_3" AS FLOAT) - 1.199333333333333) / 0.7622376689603465 AS "Feature_3" 
FROM "INPUT_DATA" AS "ADS")
 SELECT "CenteredDataForFastICA"."KEY" AS "KEY", "CenteredDataForFastICA"."Feature_0" * -1.949056546891722 + "CenteredDataForFastICA"."Feature_1" * 0.4682376457435837 + "CenteredDataForFastICA"."Feature_2" * 0.2931819865144707 + "CenteredDataForFastICA"."Feature_3" * 1.531834338344022 AS fastica_1, "CenteredDataForFastICA"."Feature_0" * 0.0365604526892373 + "CenteredDataForFastICA"."Feature_1" * 1.0730780418911068 + "CenteredDataForFastICA"."Feature_2" * 0.08716578444820255 + "CenteredDataForFastICA"."Feature_3" * 0.3683016112665801 AS fastica_2

# Execute the SQL Code

In [9]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [10]:
dataset = iris[,-5]

df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

Feature_0,Feature_1,Feature_2,Feature_3,KEY
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,2
4.7,3.2,1.3,0.2,3
4.6,3.1,1.5,0.2,4
5.0,3.6,1.4,0.2,5
5.4,3.9,1.7,0.4,6


In [11]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [12]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

KEY,fastica_1,fastica_2,fastica_3
1,-0.1747727812,0.4577064,-1.371333
2,-0.2411582808,-0.7820956,-1.298537
3,0.4278364609,-0.3034750,-1.334651
4,0.5890002061,-0.5442090,-1.271123
5,0.1680286252,0.6994855,-1.377995
6,0.0005664192,1.5671793,-1.204708


# R Caret Rpart Output

In [13]:
preprocessed  =  predict(model, iris[,-5])
df_r_out = data.frame(preprocessed)
names(df_r_out) = sprintf("Feature_%d",0:(ncol(df_r_out) - 1))

df_r_out$KEY = seq.int(nrow(dataset))

head(df_r_out)

Feature_0,Feature_1,Feature_2,KEY
-0.1747727812,0.4577064,-1.371333,1
-0.2411582808,-0.7820956,-1.298537,2
0.4278364609,-0.3034750,-1.334651,3
0.5890002061,-0.5442090,-1.271123,4
0.1680286252,0.6994855,-1.377995,5
0.0005664192,1.5671793,-1.204708,6


# Compare R and SQL output

In [14]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Feature_0,Feature_1,Feature_2,fastica_1,fastica_2,fastica_3
1,-0.1747727812,0.4577064,-1.371333,-0.1747727812,0.4577064,-1.371333
2,-0.2411582808,-0.7820956,-1.298537,-0.2411582808,-0.7820956,-1.298537
3,0.4278364609,-0.3034750,-1.334651,0.4278364609,-0.3034750,-1.334651
4,0.5890002061,-0.5442090,-1.271123,0.5890002061,-0.5442090,-1.271123
5,0.1680286252,0.6994855,-1.377995,0.1680286252,0.6994855,-1.377995
6,0.0005664192,1.5671793,-1.204708,0.0005664192,1.5671793,-1.204708


In [15]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

KEY,Feature_0,Feature_1,Feature_2,fastica_1,fastica_2,fastica_3


In [16]:
stopifnot(nrow(diffs_df) == 0)

In [17]:
summary(df_sql_out)

      KEY           fastica_1         fastica_2          fastica_3      
 Min.   :  1.00   Min.   :-2.5489   Min.   :-2.74938   Min.   :-1.5179  
 1st Qu.: 38.25   1st Qu.:-0.5225   1st Qu.:-0.57116   1st Qu.:-1.2585  
 Median : 75.50   Median : 0.1338   Median : 0.01746   Median : 0.2509  
 Mean   : 75.50   Mean   : 0.0000   Mean   : 0.00000   Mean   : 0.0000  
 3rd Qu.:112.75   3rd Qu.: 0.5237   3rd Qu.: 0.55474   3rd Qu.: 0.8915  
 Max.   :150.00   Max.   : 2.7510   Max.   : 2.80152   Max.   : 1.5723  

In [18]:
summary(df_r_out)

   Feature_0         Feature_1          Feature_2            KEY        
 Min.   :-2.5489   Min.   :-2.74938   Min.   :-1.5179   Min.   :  1.00  
 1st Qu.:-0.5225   1st Qu.:-0.57116   1st Qu.:-1.2585   1st Qu.: 38.25  
 Median : 0.1338   Median : 0.01746   Median : 0.2509   Median : 75.50  
 Mean   : 0.0000   Mean   : 0.00000   Mean   : 0.0000   Mean   : 75.50  
 3rd Qu.: 0.5237   3rd Qu.: 0.55474   3rd Qu.: 0.8915   3rd Qu.:112.75  
 Max.   : 2.7510   Max.   : 2.80152   Max.   : 1.5723   Max.   :150.00  